In [2]:
import pandas as pd
import torch
import numpy as np
from scipy.stats import spearmanr
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [3]:
log = pd.read_pickle("/home/dhvc/projetos/backup/results_full_4_255.pkl")

log = {
        'img_id': [],
        'interpretability_method': [],
        'label': [],
        'similarity_original': [],
        'similarity_perturb': [],
        'hm_original': [],
        'hm_perturbed': [],
        'alpha': [],
        'clip_model': []
    }

In [3]:
log.size

428166

In [4]:
log.head()

,img_id,interpretability_method,label,similarity_original,similarity_perturb,hm_original,hm_perturbed,alpha,clip_model
0,0,selfattn,9,0.297363,0.310791,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.015686,ViT-B/16
1,0,gradcam,9,0.297363,0.310791,"[[0.2167, 0.2167, 0.2167, 0.2167, 0.2167, 0.21...","[[0.2167, 0.2167, 0.2167, 0.2167, 0.2167, 0.21...",0.015686,ViT-B/16
2,0,maskclip,9,0.297363,0.310791,"[[0.1458, 0.1458, 0.1458, 0.1458, 0.1458, 0.14...","[[0.1458, 0.1458, 0.1458, 0.1458, 0.1458, 0.14...",0.015686,ViT-B/16
3,0,eclip,9,0.297363,0.310791,"[[0.06186, 0.06186, 0.06186, 0.06186, 0.06186,...","[[0.06186, 0.06186, 0.06186, 0.06186, 0.06186,...",0.015686,ViT-B/16
4,0,game,9,0.297363,0.310791,"[[0.01378, 0.01378, 0.01378, 0.01378, 0.01378,...","[[0.0728, 0.0728, 0.0728, 0.0728, 0.0728, 0.07...",0.015686,ViT-B/16


In [5]:
group_name = "gradcam"
filtered_group = log[log['interpretability_method'] == group_name].head()
filtered_group

,img_id,interpretability_method,label,similarity_original,similarity_perturb,hm_original,hm_perturbed,alpha,clip_model
1,0,gradcam,9,0.297363,0.310791,"[[0.2167, 0.2167, 0.2167, 0.2167, 0.2167, 0.21...","[[0.2167, 0.2167, 0.2167, 0.2167, 0.2167, 0.21...",0.015686,ViT-B/16
10,1,gradcam,9,0.284180,0.302734,"[[0.2122, 0.2122, 0.2122, 0.2122, 0.2122, 0.21...","[[0.2122, 0.2122, 0.2122, 0.2122, 0.2122, 0.21...",0.015686,ViT-B/16
19,2,gradcam,9,0.259277,0.290527,"[[0.1674, 0.1674, 0.1674, 0.1674, 0.1674, 0.16...","[[0.1674, 0.1674, 0.1674, 0.1674, 0.1674, 0.16...",0.015686,ViT-B/16
28,3,gradcam,9,0.269531,0.290771,"[[0.1893, 0.1893, 0.1893, 0.1893, 0.1893, 0.18...","[[0.1893, 0.1893, 0.1893, 0.1893, 0.1893, 0.18...",0.015686,ViT-B/16
37,4,gradcam,9,0.257324,0.299561,"[[0.2197, 0.2197, 0.2197, 0.2197, 0.2197, 0.21...","[[0.2197, 0.2197, 0.2197, 0.2197, 0.2197, 0.21...",0.015686,ViT-B/16


In [6]:
hm_original_values = []
hm_perturbed_values = []

for item in log[log['interpretability_method'] == group_name].head().iterrows():
    for tensor in item[1]['hm_original']:
        if isinstance(tensor, torch.Tensor):
            hm_original_values.extend(tensor.flatten().cpu().numpy())  # Para tensor do PyTorch
        else:
            hm_original_values.extend(tensor.flatten())  # Para arrays numpy  
    for tensor in item[1]['hm_perturbed']:
        if isinstance(tensor, torch.Tensor):
            hm_perturbed_values.extend(tensor.flatten().cpu().numpy())  # Para tensor do PyTorch
        else:
            hm_perturbed_values.extend(tensor.flatten())  # Para arrays numpy
        
    spearman_corr, _ = spearmanr(hm_original_values, hm_perturbed_values)
    print(f"Spearman rank correlation for {group_name}: {spearman_corr}")

Spearman rank correlation for gradcam: 1.0
Spearman rank correlation for gradcam: 1.0
Spearman rank correlation for gradcam: 1.0
Spearman rank correlation for gradcam: 0.9999999999999999
Spearman rank correlation for gradcam: 0.9999999999999998


In [ ]:
def calculate_spearman_for_group(group):
    hm_original_values = []
    hm_perturbed_values = []

    
    for i, row in group.iterrows():
        for tensor in row['hm_original']:
            if isinstance(tensor, torch.Tensor):
                hm_original_values.extend(tensor.flatten().cpu().numpy())  # Para tensor do PyTorch
            else:
                hm_original_values.extend(tensor.flatten())  # Para arrays numpy  
        
        for tensor in row['hm_perturbed']:
            if isinstance(tensor, torch.Tensor):
                hm_perturbed_values.extend(tensor.flatten().cpu().numpy())  # Para tensor do PyTorch
            else:
                hm_perturbed_values.extend(tensor.flatten())  # Para arrays numpy

    spearman_corr, _ = spearmanr(hm_original_values, hm_perturbed_values)
    return spearman_corr

int_method_list =  ["selfattn", "gradcam", "maskclip", "eclip", "game", "rollout", "surgery", "m2ib", "rise"]

for methd in int_method_list:
    filtered_group = log[log['interpretability_method'] == methd]
    spearman_corr = calculate_spearman_for_group(filtered_group)
    print(f"Spearman rank correlation for {methd}: {spearman_corr}")


Spearman rank correlation for selfattn: 0.9999999999999998
Spearman rank correlation for gradcam: 0.9999999999999998
Spearman rank correlation for maskclip: 0.9999999999999998
Spearman rank correlation for eclip: 1.0
Spearman rank correlation for game: 0.4786642362427312
Spearman rank correlation for rollout: 0.5125100061288176
Spearman rank correlation for surgery: 0.698657372963918
Spearman rank correlation for m2ib: 0.4995446517605409


In [4]:
def topk_intersection(map1, map2, k):
    flat1 = map1.flatten()
    flat2 = map2.flatten()
    
    # Encontrar os índices dos top-K valores
    topk_indices_1 = torch.topk(flat1.clone().detach(), k).indices
    topk_indices_2 = torch.topk(flat2.clone().detach(), k).indices

    # Converter os índices para conjuntos
    set1 = set(topk_indices_1.tolist())
    set2 = set(topk_indices_2.tolist())    
    
    # Calcular a interseção entre os conjuntos
    intersection = set1.intersection(set2)
    
    return len(intersection), len(intersection) / k

# Valor de k (ajuste conforme necessário)
k = 100

# Filtrar o dataframe para os grupos desejados
group_names = ["selfattn", "gradcam", "maskclip", "eclip", "game", "rollout", "surgery", "m2ib", "rise"]

# Iterar sobre os grupos e calcular a interseção Top-K para cada um
for group_name in group_names:
    filtered_group = log[log['interpretability_method'] == group_name]
    
    # Variáveis para calcular médias
    total_intersection_count = 0
    total_intersection_ratio = 0
    count = 0
    
    # Iterar sobre as linhas do grupo filtrado
    for idx, row in filtered_group.iterrows():
        # Converter para tensores
        map1 = torch.tensor([[t.item() for t in inner] for inner in row["hm_original"]])
        map2 = torch.tensor([[t.item() for t in inner] for inner in row["hm_perturbed"]])
        
        # Calcular a interseção
        inter_count, inter_ratio = topk_intersection(map1, map2, k)
        
        # Agregar os resultados para a média
        total_intersection_count += inter_count
        total_intersection_ratio += inter_ratio
        count += 1
    
    # Calcular a média para o grupo
    average_intersection_count = total_intersection_count / count
    average_intersection_ratio = total_intersection_ratio / count
    
    # Imprimir o resultado de Top-K para o grupo
    print(f"Top-{k} intersection count for {group_name}: {average_intersection_count} with ratio: {average_intersection_ratio:.3f}")


Top-100 intersection count for selfattn: 100.0 with ratio: 1.000
Top-100 intersection count for gradcam: 100.0 with ratio: 1.000
Top-100 intersection count for maskclip: 100.0 with ratio: 1.000
Top-100 intersection count for eclip: 100.0 with ratio: 1.000
Top-100 intersection count for game: 13.239122209610292 with ratio: 0.132
Top-100 intersection count for rollout: 18.554294362466894 with ratio: 0.186
Top-100 intersection count for surgery: 12.08399545970488 with ratio: 0.121
Top-100 intersection count for m2ib: 14.055807794173289 with ratio: 0.141
Top-100 intersection count for rise: 0.2674990541051835 with ratio: 0.003
